In [1]:
%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 17822, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 17822 (delta 15), reused 6 (delta 6), pack-reused 17794 (from 3)
Receiving objects: 100% (17822/17822), 16.96 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (12141/12141), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/yolov5')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1')

In [4]:
# 구글 드라이브의 zip 파일을 직접 /content/data 폴더에 조용히(-q) 압축 해제
!unzip -q "/content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1/tar_file/train.zip" -d /content/data
!unzip -q "/content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1/tar_file/track1_test.zip" -d /content/data

In [5]:
!rm -rf /content/data/__MACOSX

In [6]:
!ls /content/yolov5/Datasets/train | head

ls: cannot access '/content/yolov5/Datasets/train': No such file or directory


In [7]:
import json
with open("/content/data/train/01_1751_0250-1750/IR_label.json") as f:
    data = json.load(f)
print(type(data))
print(data.keys() if isinstance(data, dict) else data[:1])

<class 'dict'>
dict_keys(['exist', 'gt_rect', 'LR', 'VE', 'OC', 'FM', 'SV', 'IC', 'DBC', 'TS'])


In [8]:
data['gt_rect'][:5]

[[306, 270, 14, 11],
 [313, 269, 14, 12],
 [320, 269, 14, 13],
 [326, 269, 14, 12],
 [326, 269, 14, 12]]

In [9]:
import os
drive_path = '/content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1'
track1_test_path = '/content/data/track1_test'

print(f"Checking directory : {track1_test_path}")

if os.path.exists(track1_test_path):
    print("Directory Exists. Listing Contents.")
    contents = os.listdir(track1_test_path)
    if contents:
        for content in contents[:10]:
            print(f"- {content}")
        if len(contents) > 10:
            print(f"... and {len(contents) - 10} more items")
        else:
            print("Directory is empty")
else:
    print("Directory does not exist.")

Checking directory : /content/data/track1_test
Directory Exists. Listing Contents.
- wg2022_ir_028_split_07
- 20190926_111509_1_1
- 46_1
- new12_train_newfix
- new31_train-new
- 20190926_200510_1_7
- 3700000000002_110352_1
- 20190925_101846_1_6
- 20190926_183941_1_6
- 20190926_133516_1_3
... and 207 more items


In [10]:
# Link to Yolov5
!ln -s /content/data /content/yolov5/Datasets
!ls /content/yolov5/Datasets/train | head

01_1751_0250-1750
01_2192_0001-1500
01_4485_1167-2666
01_4485_2985-4484
02_1610_0001_1500
02_6321_0274-2773
04_8618_1215-2714
11_1
12_1
15_1


# train 구성 분석

In [11]:
import os
import json

root_dir = '/content/yolov5/Datasets/train'

length_mismatch = []
frame_mismatch = []
missing_keys = []
summary = []

for seq in sorted(os.listdir(root_dir)):
    seq_path = os.path.join(root_dir, seq)

    if not os.path.isdir(seq_path):
        continue

    json_path = os.path.join(seq_path, "IR_label.json")
    if not os.path.exists(json_path):
        continue

    with open(json_path, 'r') as f:
        data = json.load(f)

    required_keys = ["gt_rect", "exist"]
    for k in required_keys:
        if k not in data:
            missing_keys.append((seq, k))
    lengths = {k: len(v) for k, v in data.items() if isinstance(v, list)}
    if len(set(lengths.values())) != 1:
        length_mismatch.append((seq, lengths))
    image_files = [f for f in os.listdir(seq_path)
                   if f.endswith((".jpg", ".png", ".jpeg"))]

    if "gt_rect" in data:
        if len(image_files) != len(data["gt_rect"]):
            frame_mismatch.append(
                (seq, len(image_files), len(data["gt_rect"]))
            )

    if "exist" in data:
        exist_sum = sum(data["exist"])
        summary.append((seq, len(data["exist"]), exist_sum))

print("=== 검사 결과 ===")
print(f"길이 불일치 시퀀스 수: {len(length_mismatch)}")
print(f"프레임-라벨 mismatch 수: {len(frame_mismatch)}")
print(f"missing key 수: {len(missing_keys)}")

if frame_mismatch:
    print("\n[Frame mismatch 예시]")
    print(frame_mismatch[:5])

=== 검사 결과 ===
길이 불일치 시퀀스 수: 0
프레임-라벨 mismatch 수: 0
missing key 수: 0


# Track1_test 구성 분석

In [12]:
import os
import json

root_dir = '/content/yolov5/Datasets/track1_test'

length_mismatch = []
frame_mismatch = []
missing_keys = []
summary = []

for seq in sorted(os.listdir(root_dir)):
    seq_path = os.path.join(root_dir, seq)

    if not os.path.isdir(seq_path):
        continue

    json_path = os.path.join(seq_path, "IR_label.json")
    if not os.path.exists(json_path):
        continue

    with open(json_path, 'r') as f:
        data = json.load(f)

    required_keys = ["gt_rect", "exist"]
    for k in required_keys:
        if k not in data:
            missing_keys.append((seq, k))
    lengths = {k: len(v) for k, v in data.items() if isinstance(v, list)}
    if len(set(lengths.values())) != 1:
        length_mismatch.append((seq, lengths))
    image_files = [f for f in os.listdir(seq_path)
                   if f.endswith((".jpg", ".png", ".jpeg"))]

    if "gt_rect" in data:
        if len(image_files) != len(data["gt_rect"]):
            frame_mismatch.append(
                (seq, len(image_files), len(data["gt_rect"]))
            )

    if "exist" in data:
        exist_sum = sum(data["exist"])
        summary.append((seq, len(data["exist"]), exist_sum))

print("=== 검사 결과 ===")
print(f"길이 불일치 시퀀스 수: {len(length_mismatch)}")
print(f"프레임-라벨 mismatch 수: {len(frame_mismatch)}")
print(f"missing key 수: {len(missing_keys)}")

if frame_mismatch:
    print("\n[Frame mismatch 예시]")
    print(frame_mismatch[:5])

=== 검사 결과 ===
길이 불일치 시퀀스 수: 0
프레임-라벨 mismatch 수: 216
missing key 수: 216

[Frame mismatch 예시]
[('01_1667_0001-1500', 1500, 1), ('03_2499_0962-2461', 1500, 1), ('03_3780_0001-1499', 1500, 1), ('16_1', 714, 1), ('18_1', 665, 1)]


In [13]:
file_path = '/content/yolov5/Datasets/train/01_2192_0001-1500/IR_label.json'
with open(file_path, 'r') as f:
    data = json.load(f)
print(f"포함된 Key 목록 : {list(data.keys())}")
print("-" * 20)

for key in data.keys():
    length = len(data[key])
    print(f"Key : {key:10} | 데이터 개수 : {length}")
print("-" * 20)
print("첫 번째 프레임 (Index 0)의 상태 : ")
print(f"  - 존재 여부 (Exist) : {data['exist'][0]}")
print(f"  - 좌표 (gt_rect) : {data['gt_rect'][0]}")
print(f"  - 가림 여부 (OC) : {data['OC'][0]}")

포함된 Key 목록 : ['exist', 'gt_rect', 'LR', 'VE', 'OC', 'FM', 'SV', 'IC', 'DBC', 'TS']
--------------------
Key : exist      | 데이터 개수 : 1500
Key : gt_rect    | 데이터 개수 : 1500
Key : LR         | 데이터 개수 : 1500
Key : VE         | 데이터 개수 : 1500
Key : OC         | 데이터 개수 : 1500
Key : FM         | 데이터 개수 : 1500
Key : SV         | 데이터 개수 : 1500
Key : IC         | 데이터 개수 : 1500
Key : DBC        | 데이터 개수 : 1500
Key : TS         | 데이터 개수 : 1500
--------------------
첫 번째 프레임 (Index 0)의 상태 : 
  - 존재 여부 (Exist) : 1
  - 좌표 (gt_rect) : [268, 279, 29, 22]
  - 가림 여부 (OC) : 0


# train 데이터 분석

In [14]:
root_dir_check = '/content/yolov5/Datasets/train'

missing_json=[]
missing_gt_rect = []
valid_folders= []
image_scale = {}

for folder in sorted(os.listdir(root_dir_check)):
    folder_path = os.path.join(root_dir_check, folder)

    if not os.path.isdir(folder_path):
        continue

    json_path = os.path.join(folder_path, "IR_label.json")
    if not os.path.exists(json_path):
        missing_json.append(folder)
        continue

    try:
        with open(json_path, "r") as f:
            data = json.load(f)
        if "gt_rect" not in data:
            missing_gt_rect.append(folder)
        else:
            valid_folders.append(folder)
    except Exception as e:
        print(f"[ERROR] - {folder} : {e}")

    files = os.listdir(folder_path)
    files = [f for f in os.listdir(folder_path) if f.endswith((".jpg", ".png", ".jpeg"))]
    scale = len(files)

    if scale not in image_scale:
        image_scale[scale] = 1
    else:
        image_scale[scale] = image_scale[scale]+1


print("=== 검사 결과 ===")
print(f"total number of folders : {len(os.listdir(root_dir_check))}")
print(f"Normal (gt_rect extist) : {len(valid_folders)}")
print(f"Missing IR_label.json : {len(missing_json)}")
print(f"No gt_rect label.jsons : {len(missing_gt_rect)}")
print(f"Length of each train Folder : {image_scale}")
print(f"Sum of dictionary Values : {sum(image_scale.values())}")

print(f"\n\n=== Total {len(os.listdir(root_dir_check)) - len(valid_folders)} Abnormal Case ===")
for item in os.listdir(root_dir_check):
    if not os.path.isdir(os.path.join(root_dir_check, item)):
        print("Not Directory : ", item)

=== 검사 결과 ===
total number of folders : 225
Normal (gt_rect extist) : 223
Missing IR_label.json : 0
No gt_rect label.jsons : 0
Length of each train Folder : {1500: 78, 1467: 1, 487: 1, 714: 1, 619: 2, 1000: 73, 948: 2, 989: 2, 923: 1, 404: 1, 858: 1, 920: 1, 549: 1, 926: 1, 932: 1, 950: 1, 921: 1, 889: 1, 956: 1, 944: 1, 959: 1, 942: 2, 851: 1, 937: 1, 857: 1, 938: 1, 898: 1, 958: 1, 930: 1, 961: 1, 924: 1, 892: 1, 929: 1, 907: 1, 794: 1, 933: 1, 977: 1, 979: 1, 986: 1, 987: 2, 429: 2, 509: 1, 139: 1, 28: 1, 163: 1, 919: 1, 818: 1, 646: 1, 820: 1, 790: 1, 537: 1, 666: 1, 728: 1, 730: 1, 744: 1, 679: 1, 997: 1, 604: 1, 1630: 1, 774: 1, 519: 1, 580: 1, 430: 1, 470: 1, 450: 1, 590: 1, 1491: 1, 850: 1}
Sum of dictionary Values : 223


=== Total 2 Abnormal Case ===
Not Directory :  list.txt
Not Directory :  .DS_Store


# track1_test 분석

In [15]:
check_test_dir = '/content/yolov5/Datasets/track1_test'


missing_json=[]
missing_gt_rect = []
valid_folders= []
image_scale = {}

for folder in sorted(os.listdir(check_test_dir)):
    folder_path = os.path.join(check_test_dir, folder)

    if not os.path.isdir(folder_path):
        continue

    json_path = os.path.join(folder_path, "IR_label.json")
    if not os.path.exists(json_path):
        missing_json.append(folder)
        continue

    try:
        with open(json_path, "r") as f:
            data = json.load(f)
        if "gt_rect" not in data:
            missing_gt_rect.append(folder)
        else:
            valid_folders.append(folder)
    except Exception as e:
        print(f"[ERROR] - {folder} : {e}")

    files = os.listdir(folder_path)
    files = [f for f in os.listdir(folder_path) if f.endswith((".jpg", ".png", ".jpeg"))]
    scale = len(files)

    if scale not in image_scale:
        image_scale[scale] = 1
    else:
        image_scale[scale] = image_scale[scale]+1


print("=== 검사 결과 ===")
print(f"total number of folders : {len(os.listdir(check_test_dir))}")
print(f"Normal (gt_rect extist) : {len(valid_folders)}")
print(f"Missing IR_label.json : {len(missing_json)}")
print(f"No gt_rect label.jsons : {len(missing_gt_rect)}")
print(f"Length of each train Folder : {image_scale}")
print(f"Sum of dictionary Values : {sum(image_scale.values())}")

print(f"\n\n=== Total {len(os.listdir(check_test_dir)) - len(valid_folders)} Abnormal Case ===")
for item in os.listdir(check_test_dir):
    if not os.path.isdir(os.path.join(check_test_dir, item)):
        print("Not Directory : ", item)

=== 검사 결과 ===
total number of folders : 217
Normal (gt_rect extist) : 216
Missing IR_label.json : 0
No gt_rect label.jsons : 0
Length of each train Folder : {1500: 55, 714: 1, 665: 1, 1124: 1, 1000: 99, 899: 1, 348: 1, 164: 1, 869: 1, 658: 1, 379: 1, 534: 1, 604: 1, 979: 1, 914: 1, 929: 1, 943: 2, 939: 1, 910: 1, 924: 1, 935: 1, 930: 1, 39: 1, 934: 2, 898: 1, 894: 1, 919: 1, 942: 1, 900: 1, 938: 2, 706: 1, 926: 1, 948: 1, 688: 1, 764: 1, 313: 1, 364: 1, 299: 1, 174: 1, 493: 1, 564: 1, 1227: 1, 976: 1, 542: 1, 1374: 1, 759: 1, 566: 1, 742: 1, 813: 1, 756: 1, 711: 1, 600: 1, 410: 1, 450: 1, 1491: 7}
Sum of dictionary Values : 216


=== Total 1 Abnormal Case ===
Not Directory :  .DS_Store


In [16]:
from __future__ import absolute_import
import numpy as np
import cv2
from tqdm import tqdm
import json
import glob
import os

from detection_siamfc import TrackerSiamFC

def iou(bbox1, bbox2):
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, w1_1, h1_1) = bbox1
    (x0_2, y0_2, w1_2, h1_2) = bbox2
    x1_1 = x0_1 + w1_1
    x1_2 = x0_2 + w1_2
    y1_1 = y0_1 + h1_1
    y1_2 = y0_2 + h1_2

    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <=0:
        return 0

    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)

    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [17]:
def not_exist(pred):
    return (len(pred) == 1 and pred[0] == 0) or (len(pred) == 0)

In [18]:
def eval(out_res, label_res):
    measure_per_frame = []
    penalty_measure = []
    for _pred, _gt, _exist in zip(out_res, label_res['gt_rect'], label_res['exist']):
        # Target 존재 x : _exist==False | Target 존재 O : _exist == True
        measure_per_frame.append(not_exist(_pred) if not _exist else iou(_pred, _gt) if len(_pred) > 1 else 0)
        if _exist:
            if (len(_pred) > 1 and iou(_pred, _gt) > 1e-5):
                penalty_measure.append(0)
            else:
                penalty_measure.append(1)
    if len(measure_per_frame) == 0:
        measure_per_frame_mean = 0
    else:
        measure_per_frame_mean = np.mean(measure_per_frame)

    if len(penalty_measure) == 0:
        penalty_measure_mean = 0
    else:
        penalty_measure_mean = np.mean(penalty_measure)

    return measure_per_frame_mean - 0.2 * (penalty_measure_mean ** 0.3)

In [34]:
import os
import cv2
import json
import glob
import torch
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import clear_output

def main(mode='IR', visualization=False, target_folders=None):
    # Lucas-Kanade Optical Flow Parameter settings.
    # == Lucas-Kanade operate flow ==
    # p0 (prev frame)  -> calcOpticalFlowPyrLK -> p1(current frame position)
    # p0 : feature coordinate on prev_frame  |   p1 : position of the feature at current frame
    # and estimae camera motion matrix by using ` M = estimateAffinePartial2D(good_old, good_new)`
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
    )
    global drive_path
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"현재 사용 장치 : {device}")

    net_path = os.path.join(drive_path, "Baseline_code", 'model.pth')
    tracker = TrackerSiamFC(net_path=net_path)
    yolo_model = tracker.initialize_yolo()
    yolo_model.to(device)
    tracker.net.to(device)

    #video_paths = glob.glob('/content/data/track1_test/*')
    #video_paths = video_paths[:3]

    base_dir = '/content/data/track1_test'

    # target_folders=[
    #     "20190925_130434_1_6",
    #     "20190925_152412_1_7",
    #     "20190926_193515_1_2",
    #     "new40_train-new",
    #     "20190925_111757_1_3"
    # ]
    if target_folders is None:
        video_paths = sorted([os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    else:
        video_paths = sorted([os.path.join(base_dir, d) for d in target_folders if os.path.isdir(os.path.join(base_dir, d))])

    output_dir = os.path.join('results', tracker.name)
    os.makedirs(output_dir, exist_ok=True)

    local_output_dir = '/content/data'

    overall_success = []

    #
    print(f"Total Video nums : {len(video_paths)}")

    for video_id, video_path in enumerate(video_paths, start=1):
        print(f"\n===== {video_id}/{len(video_paths)} =====")
        gmc_initialized = False
        track_active = False
        prev_gray = None
        p0 = None
        video_preds = [] # for evaluate
        video_name = os.path.basename(video_path)
        frame_files = sorted([
            f for f in os.listdir(video_path)
            if f.endswith(('.jpg', '.png', '.jpeg'))
        ])
        res_file = os.path.join(video_path, 'IR_label.json')
        with open(res_file, 'r') as f:
            label_res = json.load(f)

        if 'exist' not in label_res:
            label_res['exist'] = [1] * len(label_res['gt_rect'])

        gt_rects = label_res['gt_rect']
        exist_flags = label_res['exist']
        video_ious = []

        # Calculate num_frames to prevent IndexError
        if len(gt_rects) > 1: # 꼭!!
            num_frames = min(len(frame_files), len(gt_rects))
        else:
            num_frames = len(frame_files)

        ########
        if visualization and len(frame_files) > 0:
            first_frame_path = os.path.join(video_path, frame_files[0])
            first_frame = cv2.imread(first_frame_path)
            h, w = first_frame.shape[:2]

            out_video_path = os.path.join(local_output_dir, f"{video_name}_tracked.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writer = cv2.VideoWriter(out_video_path, fourcc, 30.0, (w, h))
        ########

        for frame_id in range(num_frames):
            frame_file = frame_files[frame_id]
            frame_path = os.path.join(video_path, frame_file)
            frame = cv2.imread(frame_path)
            curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            im_vis = frame.copy()

            if not track_active: # Tracking 할 것 없음
                pred_bbox, im_vis = tracker.init(frame, yolo_model)
                if len(pred_bbox) > 1:
                    track_active = True
                    gmc_initialized = False
                else:
                    pred_bbox = [0]
            else:
                pred_bbox = tracker.update(frame)

                if len(pred_bbox) == 1:
                    track_active = False
                    gmc_initialized = False
                    pred_bbox = [0]
            if track_active and not gmc_initialized:

                prev_gray = curr_gray.copy()
                p0 = cv2.goodFeaturesToTrack(prev_gray, 80, 0.01, 7, blockSize=7)
                gmc_initialized = True

            elif track_active and gmc_initialized:

                if p0 is not None and len(p0) > 15:
                  # p1 : p0에 있던 점들이 현재 프레임에서 이사 간 새로운 좌표들.
                    p1, st, err = cv2.calcOpticalFlowPyrLK(
                        prev_gray, curr_gray, p0, None, **lk_params)
                    good_new = p1[st == 1]
                    good_old = p0[st == 1]
                    if len(good_old) > 15:
                        M, _ = cv2.estimateAffinePartial2D(
                            good_old, good_new,
                            method=cv2.RANSAC,
                            ransacReprojThreshold=3)

                        if M is not None:

                            h_img, w_img = curr_gray.shape
                            warped_prev = cv2.warpAffine(prev_gray, M, (w_img, h_img))
                            residual = cv2.absdiff(warped_prev, curr_gray)

                            x, y, w, h = map(int, pred_bbox)
                            x = max(0, x)
                            y = max(0, y)
                            w = min(w, w_img - x)
                            h = min(h, h_img - y)

                            inside = residual[y:y+h, x:x+w]
                            inside_score = np.mean(inside)

                            outside_mask = np.ones_like(residual, dtype=np.uint8)
                            outside_mask[y:y+h, x:x+w] = 0
                            outside_score = np.mean(residual[outside_mask == 1])

                            ratio = inside_score / (outside_score + 1e-6)

                            #if ratio < 1.1:
                            if ratio < 0.001:
                                track_active = False
                                gmc_initialized = False
                                pred_bbox = [0]

                    if len(good_new) > 0:
                        p0 = good_new.reshape(-1, 1, 2)

                if p0 is None or len(p0) < 20:
                    p0 = cv2.goodFeaturesToTrack(prev_gray, 80, 0.01, 7, blockSize=7)

            prev_gray = curr_gray.copy()

            if frame_id < len(gt_rects):
                if len(pred_bbox) > 1:
                    video_preds.append(pred_bbox)
                else:
                    video_preds.append([0])

            # Ensure frame_id is within bounds of gt_rects and exist_flags
            if frame_id < len(gt_rects) and exist_flags[frame_id] == 1:

                if len(pred_bbox) > 1:
                    iou_score = iou(pred_bbox, gt_rects[frame_id])
                else:
                    iou_score = 0.0

                video_ious.append(iou_score)

            # Visualization
            if visualization:
                if len(pred_bbox) > 1:
                    x, y, w, h = map(int, pred_bbox)
                    cv2.rectangle(im_vis, (x,y), (x+w, y+h), (0, 0, 255), 2)
                video_writer.write(im_vis)

                ###
                #clear_output(wait=True)
                #cv2_imshow(im_vis)
                ###
        if visualization:
            video_writer.release()
            print(f"저장 완료 {out_video_path}")
            mac_video_path = os.path.join(local_output_dir, f"{video_name}_mac_ready.mp4")
            os.system(f"ffmpeg -y -hide_banner -loglevel error -i '{out_video_path}' -vcodec libx264 '{mac_video_path}'")
            print(f"저장 완료 : {mac_video_path}")


        if len(video_ious) > 0:
            #video_success = np.mean(video_ious)
            # overall_success.append(video_success)
            # print(f"{video_name} Success: {video_success:.4f}")
            video_aoa = eval(video_preds, label_res)
            overall_success.append(video_aoa)
            print(f"{video_name} AOA : {video_aoa:.4f}")
    if len(overall_success) > 0:
        final_score = np.mean(overall_success)
        print("\n===============================")
        print(f"Overall Success: {final_score:.4f}")
        print("===============================")

# if __name__ == '__main__':
#     main(mode='IR', visualization=True)

In [39]:
# 전체 수행
main(mode='IR', visualization=False)

# Target 수행
targets = [
        "20190925_130434_1_6",
]

#main(mode='IR', visualization=False, target_folders=targets)

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU



현재 사용 장치 : cuda:0


Fusing layers... 
YOLOv5s_s summary: 220 layers, 7926792 parameters, 0 gradients
YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU



Total Video nums : 216

===== 1/216 =====


YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU



...

wg2022_ir_054_split_01 AOA : 0.3704

===== 215/216 =====


YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 

wg2022_ir_054_split_02 AOA : -0.2000

===== 216/216 =====


YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 CPU

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.10.0+cu128 

wg2022_ir_054_split_03 AOA : 0.4341

Overall Success: 0.6067


In [ ]:
!ls -lh /content/data/*.mp4

-rw-r--r-- 1 root root 1.4M Feb 21 15:46 /content/data/03_2499_0962-2461_mac_ready.mp4
-rw-r--r-- 1 root root 2.7M Feb 21 15:46 /content/data/03_2499_0962-2461_tracked.mp4
-rw-r--r-- 1 root root 1.4M Feb 21 15:47 /content/data/20190925_101846_1_4_mac_ready.mp4
-rw-r--r-- 1 root root 2.7M Feb 21 15:47 /content/data/20190925_101846_1_4_tracked.mp4
-rw-r--r-- 1 root root 564K Feb 21 15:49 /content/data/46_1_mac_ready.mp4
-rw-r--r-- 1 root root 886K Feb 21 15:49 /content/data/46_1_tracked.mp4


In [ ]:
import os

source_dir = os.path.join('results', 'Yolo_SiamFC')

print("Source path:", source_dir)
print("Exists:", os.path.exists(source_dir))
print("Files:", os.listdir(source_dir))

Source path: results/Yolo_SiamFC
Exists: True
Files: []


In [ ]:
import os
import shutil
import datetime

source_dir = os.path.join('results', 'Yolo_SiamFC')

timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
backup_dir = os.path.join(drive_path, f'Yolo_SiamFC_{timestamp}')

print(f"Backing up to: {backup_dir}")

shutil.copytree(source_dir, backup_dir)

print("Backup finished successfully.")

Backing up to: /content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1/Yolo_SiamFC_20260221_154923
Backup finished successfully.


# 구글 드라이브에서 가져와 영상으로..

해당 로직은 `main`을 local에 영상을 저장하도록 재설계하여 불필요합니다.

In [ ]:
import cv2
import json
import os
import glob

# 1. 설정
#video_name = "1_1" # (혹은 37000... 등 실제 폴더명)
#video_name = "20190925_101846_1_4"
video_name = "03_2499_0962-2461"
drive_path = "/content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1"

image_folder = f"/content/data/track1_test/{video_name}"
#result_path = f"{drive_path}/results_backup/Yolo_SiamFC/{video_name}.txt"
result_path = f"{drive_path}/results/Yolo_SiamFC/per_video_outputs/{video_name}.txt"
output_video_path = f"{drive_path}/vis_result_{video_name}.mp4"

image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')

if not os.path.exists(result_path):
    print(f"There is no result file: {result_path}")
else:
    with open(result_path, 'r') as f:
        data = json.load(f)
        predictions = data['res'] # [[x,y,w,h], ...]

    image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(image_extensions)])

    if len(image_files) == 0:
        print(f"There is no single image. Check it out at : {image_folder}")
    else:
        print(f"총 {len(image_files)}장의 이미지를 찾았습니다.")

        # 첫 번째 이미지를 읽어서 영상 크기 결정
        first_frame = cv2.imread(image_files[0])
        height, width, layers = first_frame.shape

        # 비디오 저장 설정 (FPS는 30으로 설정)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, 30, (width, height))

        # 4. 한 장씩 읽으면서 박스 그리고 영상으로 저장
        for i, img_path in enumerate(image_files):
            frame = cv2.imread(img_path)

            if frame is None:
                continue

            # 예측 좌표가 있으면 그리기
            if i < len(predictions):
                bbox = predictions[i]
                if len(bbox) == 4: # [x, y, w, h] 형태일 때만
                    x, y, w, h = map(int, bbox)
                    # 파란색 박스 (BGR: Blue)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    cv2.putText(frame, f"Frame: {i}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            out.write(frame)

            if i % 100 == 0:
                print(f"... {i}번째 프레임 처리 중")

        out.release()
        print(f"\nSaved Visualization Video ")
        print(f"저장 경로: {output_video_path}")

총 1500장의 이미지를 찾았습니다.
... 0번째 프레임 처리 중
... 100번째 프레임 처리 중
... 200번째 프레임 처리 중
... 300번째 프레임 처리 중
... 400번째 프레임 처리 중
... 500번째 프레임 처리 중
... 600번째 프레임 처리 중
... 700번째 프레임 처리 중
... 800번째 프레임 처리 중
... 900번째 프레임 처리 중
... 1000번째 프레임 처리 중
... 1100번째 프레임 처리 중
... 1200번째 프레임 처리 중
... 1300번째 프레임 처리 중
... 1400번째 프레임 처리 중

Saved Visualization Video 
저장 경로: /content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1/vis_result_03_2499_0962-2461.mp4


In [ ]:
valid_boxes = [b for b in predictions if len(b) == 4]
print("유효 박스 개수:", len(valid_boxes))

유효 박스 개수: 1


In [ ]:
print("len:", len(predictions))
print("sample:", predictions[:10])
print("valid:", len([b for b in predictions if len(b)==4]))

len: 1
sample: [[202.0, 254.0, 14.0, 10.0]]
valid: 1
